### Simulation

* make 4 sets of sequences: 250 seq , 1000 seqs, 5000 seqs, each 250 bp (meme takes 60000 only)
    * Use random generator with real frequencies
* Insert real motifs but have been modified to have mC.
    * CEBPB , NRF1, SP1, CTCF, made up motif
* Different abundances: 1%, 2%, 5%, 10%, 25% of peaks
    

In [17]:
import os 
from sys import argv
import random as rd
from copy import deepcopy

def load_motifsEF(filename):
    file=open(filename)
    seq=file.read().split("MOTIF")
    seq=seq[1:]
    motifs={}
    infos={}
    for s in range(len(seq)):
        t=seq[s].strip().split("\n")
        motifs[t[0]]=t[2:]  #remove HOCOMOCO URL
        infos[t[0].split('_')[0]]=t[:2]
    for m in motifs:
        tdict={'A':[],'C':[],'G':[],'T':[],'E':[],'F':[]}
        for pos in range(len(motifs[m])):
            tmp=motifs[m][pos].strip().split("\t")
            tdict['A']+=[float(tmp[0])]
            tdict['C']+=[float(tmp[1])]
            tdict['G']+=[float(tmp[2])]
            tdict['T']+=[float(tmp[3])]
            tdict['E']+=[float(tmp[4])]
            tdict['F']+=[float(tmp[5])]
        motifs[m]=tdict
    
    return motifs,infos


def load_motifs(filename):
    file=open(filename)
    seq=file.read().split("MOTIF")
    seq=seq[1:]
    motifs={}
    infos={}
    for s in range(len(seq)):
        t=seq[s].strip().split("\n")
        motifs[t[0]]=t[2:-1]  #remove HOCOMOCO URL
        infos[t[0].split('_')[0]]=t[:2]
    for m in motifs:
        tdict={'A':[],'C':[],'G':[],'T':[]}
        for pos in range(len(motifs[m])):
            tmp=motifs[m][pos].strip().split("\t")
            tdict['A']+=[float(tmp[0])]
            tdict['C']+=[float(tmp[1])]
            tdict['G']+=[float(tmp[2])]
            tdict['T']+=[float(tmp[3])]
            #tdict['E']+=[float(tmp[4])]
            #tdict['F']+=[float(tmp[5])]
        motifs[m]=tdict
    
    return motifs,infos
def insertBaseToPWM(pwm,loc,prop):
    #randomly insert modified bases to an existing pwm
    #insert at loc location, probability is prop
    newpwm = deepcopy(pwm)
    k = len(pwm['A'])
    base = rd.choice(['E','E']) # only adds E, for this is typeE
    freq = prop #rd.choice(range(5,10))*0.1 #prop 0.5 - 1.0
    print base,loc,freq
    newpwm['E'] = [0]*k #add other empty columns to the matrix
    newpwm['F'] = [0]*k

    for c in newpwm.keys():
        newpwm[c][loc] = (1-freq)/(len(newpwm.keys())-1)
    newpwm[base][loc] = freq
    return newpwm

def insertBaseToPWM_typeE(pwm,loc,props):
    #randomly insert modified bases to an existing pwm
    #insert at loc location, probability is prop
    newpwm = deepcopy(pwm)
    k = len(pwm['A'])
    newpwm['E'] = [0]*k #add another empty column to the matrix
    
    for c in newpwm.keys():
        newpwm[c][loc] = props[c]
    return newpwm

def randomkmer_typeEF(PWM):
    #take a PWM and randomly create a Kmer based on it
    # chance goes A->C->G->T->E->F
    kmer=""
    for pos in range(len(PWM['A'])):
        die=rd.randint(0,100000)
        if die<=PWM['A'][pos]*100000:
            kmer+='A'
        elif die<=(PWM['A'][pos]+PWM['C'][pos])*100000:
            kmer+='C'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos])*100000:
            kmer+='G'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos]+PWM['T'][pos])*100000:
            kmer+='T'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos]+PWM['T'][pos]+PWM['E'][pos])*100000:
            kmer+='E'
        else:
            kmer+='F'
    return kmer

def loadFasta(filename):
    seqs = {}
    lines = open(filename).read().split('>')[1:]
    for line in lines:
        tmp = line.split('\n')
        seqs[tmp[0]] = ''.join(tmp[1:])
    return seqs

def insertoseqs(pwm,seqs,numseqtopick):
    #take pwm, the seqs, and insert the pwm to numseqtopick of the seqs 
    
    seqspicked = rd.sample(seqs.keys(),numseqtopick)
    #print seqspicked
    for s in seqspicked:
        kmer = randomkmer_typeEF(pwm)
        #print kmer
        string = list(seqs[s])
        insertloc = rd.randrange(0,len(string)-len(kmer))
        #print insertloc
        for i in xrange(len(kmer)):
            string[insertloc + i]=kmer[i]
    
        seqs[s] = ''.join(string)
    return seqs
def writeoutPWM(pwm,name,outfile):
    target = open(outfile,'w')
    target.write("MOTIF\t"+name+'\n')
    for loc in range(len(pwm['A'])):
        line = []
        for base in ['A','C','G','T','E','F']:
            if base not in pwm: continue
            line += [str(pwm[base][loc])]
        line = '\t'.join(line)
        target.write(line+'\n')
    target.close()
    return
def printPWM(pwm):
    print '\t'.join(['A','C','G','T','E','F'])
    for loc in range(len(pwm['A'])):
        line = []
        for base in ['A','C','G','T','E','F']:
            if base in pwm:
                line += [str(round(pwm[base][loc],3))]
        print '\t'.join(line)
    return
def getConsensus(pwm):
    #take the dict pwm and print the consensus of the pwm
    cnss = '' 
    for loc in range(len(pwm['A'])):
        maxbase = 'A'
        maxvalue = 0.0
        for base in pwm:
            if pwm[base][loc] > maxvalue:
                maxbase = base
                maxvalue = pwm[base][loc]
        cnss += maxbase
    return cnss

In [3]:
knownMotifs,knownInfos = load_motifs("HOCOMOCOv11_core_HUMAN_mono_meme_format.meme")

In [4]:
mymotifs = {}
for key in knownMotifs:
    if "CTCF_" in key or "CEBPB" in key or "NRF1" in key:
        print key
        #print knownMotifs[key]
        print getConsensus(knownMotifs[key])
        mymotifs[key.split('_')[0]] = knownMotifs[key]
        

#add the methylation to these motifs
#CEBPB: no. 5
props = {
    'A':0.024376,
    'C':0.085981,
    'G':0.014709,
    'T':0.214204,
    'E':0.660729
        }
loc = 5 
mymotifs['CEBPB'] = insertBaseToPWM_typeE(loc=loc, props=props, pwm=mymotifs['CEBPB'])

#CTCF: loc = 11
loc = 11
props = {
    'A':0.0,
    'C':0.2,
    'G':0.0,
    'T':0.0,
    'E':0.8
        }

mymotifs['CTCF'] = insertBaseToPWM_typeE(loc=loc, props=props, pwm=mymotifs['CTCF'])

#NRF1: loc = 11
loc = 11
props = {
    'A':0.0,
    'C':0.2,
    'G':0.0,
    'T':0.0,
    'E':0.8
        }

mymotifs['NRF1'] = insertBaseToPWM_typeE(loc=loc, props=props, pwm=mymotifs['NRF1'])
#add the gatEGaca motif too 
gataEGca = {
    'A':[0.1,0.7,0.0,0.8,0.0,0.0,0.1,0.9],
    'C':[0.0,0.1,0.1,0.2,0.2,0.0,0.9,0.0],
    'G':[0.9,0.0,0.0,0.0,0.0,0.8,0.0,0.1],
    'T':[0.0,0.1,0.9,0.0,0.0,0.1,0.0,0.0],
    'E':[0.0,0.1,0.0,0.0,0.8,0.1,0.0,0.0],
}
print getConsensus(gataEGca)
mymotifs['gataEGca'] = gataEGca


CTCF_HUMAN.H11MO.0.A
TGGCCACCAGGGGGCGCCA
CEBPB_HUMAN.H11MO.0.A
GATTGTGCAATA
NRF1_HUMAN.H11MO.0.A
CACTGCGCATGCGCAGG
GATAEGCA


In [5]:
for key in mymotifs:
    print key,getConsensus(mymotifs[key])
    printPWM(mymotifs[key])
     

CEBPB GATTGEGCAATA
A	C	G	T	E	F
0.238	0.099	0.352	0.311	0.0
0.454	0.132	0.388	0.026	0.0
0.0	0.002	0.002	0.996	0.0
0.0	0.0	0.3	0.7	0.0
0.302	0.002	0.513	0.183	0.0
0.024	0.086	0.015	0.214	0.661
0.0	0.0	1.0	0.0	0.0
0.081	0.899	0.0	0.02	0.0
1.0	0.0	0.0	0.0	0.0
1.0	0.0	0.0	0.0	0.0
0.0	0.244	0.148	0.608	0.0
0.37	0.333	0.183	0.115	0.0
gataEGca GATAEGCA
A	C	G	T	E	F
0.1	0.0	0.9	0.0	0.0
0.7	0.1	0.0	0.1	0.1
0.0	0.1	0.0	0.9	0.0
0.8	0.2	0.0	0.0	0.0
0.0	0.2	0.0	0.0	0.8
0.0	0.0	0.8	0.1	0.1
0.1	0.9	0.0	0.0	0.0
0.9	0.0	0.1	0.0	0.0
CTCF TGGCCACCAGGEGGCGCCA
A	C	G	T	E	F
0.094	0.36	0.126	0.42	0.0
0.152	0.168	0.522	0.158	0.0
0.258	0.084	0.554	0.104	0.0
0.068	0.856	0.036	0.04	0.0
0.002	0.996	0.002	0.0	0.0
0.7	0.006	0.242	0.052	0.0
0.016	0.652	0.322	0.01	0.0
0.098	0.594	0.074	0.234	0.0
0.906	0.018	0.054	0.022	0.0
0.002	0.002	0.992	0.004	0.0
0.386	0.0	0.612	0.002	0.0
0.0	0.2	0.0	0.0	0.8
0.012	0.002	0.984	0.002	0.0
0.05	0.026	0.86	0.064	0.0
0.068	0.876	0.012	0.044	0.0
0.396	0.014	0.584	0.006	0.0
0.076	0.562	0.32

### Make sequences randomly

In [6]:
import random
import itertools
import bisect
import numpy as np

def randomseqDimer(props,length):
    """
    Generate string randomly at certain length
    - props: a dict of dimers probability for DNA 
    
    """
    #choices, weights = fprob_table.items()
    fprob_table = props
    K = length
    choices = []
    weights = []
    for key in fprob_table:
        choices += [key]
        weights += [fprob_table[key]]
    #cumdist = list(itertools.accumulate(weights))
    cumdist = []
    cumprop = 0.0
    for i in range(len(choices)):
        cumprop += weights[i]
        cumdist += [cumprop]
    results = []
    s = ""
    while len(s) < K:
        x = random.random() * cumdist[-1]
        s += choices[bisect.bisect(cumdist, x)]
        #print s

    return s

In [7]:
### getting the dimer props 

backgroundfile = "../test_data_typeE/background_typeE-5.tsv"
lines = open(backgroundfile).readlines()

dimercount = {}
for line in lines[1:]:
    #print line.strip()
    tmp = line.strip().split()
    seq = tmp[0]
    k = 2 
    multiplier = int(tmp[1])
    for i in range(len(seq) - k + 1):
        dimer = seq[i:i+k]
        try: 
            dimercount[dimer] += multiplier*1
        except KeyError:
            dimercount[dimer] = multiplier*1 
dimerProps = {}
total = 0.0
for key in dimercount:
    total += dimercount[key]
for key in dimercount:
    dimerProps[key] = dimercount[key]/total
#dimerProps

In [14]:
abundances = [0.01,0.02,0.05,0.1,0.25]
sizes = [250, 1000, 5000]
seq_length_avg = 250
seq_length_std = 50

resultdir = './files/SIM_seqs/'
for motif in mymotifs:
    print motif
    seqsets = {} 
    for a in abundances:
        for siz in sizes:
            name = str(a) + '_' + str(siz)
            print name
            outfile = open(resultdir + motif+'_'+name+'.faa','w')
            
            seq_set = {}
            for i in range(siz):
                seqlength = int(np.random.normal(scale=seq_length_std, loc=seq_length_avg))
                seq = randomseqDimer(props=dimerProps,length=seqlength)
                seq_set[i] = seq 
            # now , insert the motif to the seqs
            numtoinsert = int(a*siz) + 1
            print numtoinsert
            seq_set = insertoseqs(numseqtopick=numtoinsert,pwm=mymotifs['gataEGca'],seqs=seq_set)
            
            #write out the fasta file 
            for key in seq_set:
                line = ">"+str(key)+'\n'+seq_set[key]
                outfile.write(line+'\n')
            outfile.close()
        
        


CEBPB
0.01_250
3
0.01_1000
11
0.01_5000
51
0.02_250
6
0.02_1000
21
0.02_5000
101
0.05_250
13
0.05_1000
51
0.05_5000
251
0.1_250
26
0.1_1000
101
0.1_5000
501
0.25_250
63
0.25_1000
251
0.25_5000
1251
gataEGca
0.01_250
3
0.01_1000
11
0.01_5000
51
0.02_250
6
0.02_1000
21
0.02_5000
101
0.05_250
13
0.05_1000
51
0.05_5000
251
0.1_250
26
0.1_1000
101
0.1_5000
501
0.25_250
63
0.25_1000
251
0.25_5000
1251
CTCF
0.01_250
3
0.01_1000
11
0.01_5000
51
0.02_250
6
0.02_1000
21
0.02_5000
101
0.05_250
13
0.05_1000
51
0.05_5000
251
0.1_250
26
0.1_1000
101
0.1_5000
501
0.25_250
63
0.25_1000
251
0.25_5000
1251
NRF1
0.01_250
3
0.01_1000
11
0.01_5000
51
0.02_250
6
0.02_1000
21
0.02_5000
101
0.05_250
13
0.05_1000
51
0.05_5000
251
0.1_250
26
0.1_1000
101
0.1_5000
501
0.25_250
63
0.25_1000
251
0.25_5000
1251


In [13]:
for motif in mymotifs:
    print motif

CEBPB
gataEGca
CTCF
NRF1


In [20]:
for m in mymotifs:
    writeoutPWM(name='1',outfile=m+".SIM.typeE.meme",pwm=mymotifs[m])